# Vizualiziranje netrenirane mreže

Neki rezultati u vizualizaciji _kitty_ mreže su mi, well, bili _čudni_, odnosno činilo se da neuroni pokazuju više-nego-što-bih-očekivao pravilnosti u checkpointu _prije ikakvog treniranja_. Zato ovdje ide svježe-učitana mreža pod skalpel, s težinama određenima s tim koja je je već defaultna PyTorch metoda inicijalizacije.

Sav kod i metode su samo copy-paste iz _kitty_ vizualizacija bilježnice, a čak je i mreža copy-pasteana, samo ovdje nisam ovdje učitavao nikakve _checkpointe_.

## Definicije i importovi

In [ ]:
pip install torch-lucent

In [ ]:
from lucent.optvis.transform import pad, jitter, random_rotate, random_scale
from lucent.optvis import render, param, transform, objectives
from tqdm import tqdm
import numpy as np

In [ ]:
import torch

from lucent.optvis import render, param, transform, objectives

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# da bilježnice budu manje:
%config InlineBackend.figure_format = 'jpg'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0 = nn.BatchNorm2d(3)
        self.conv1 = nn.Conv2d(3, 9, 3)
        self.pool1 = nn.AvgPool2d(4, 4)
        
        self.conv1_bn = nn.BatchNorm2d(9)
        self.conv2 = nn.Conv2d(9, 16, 3)
        self.pool2 = nn.AvgPool2d(4, 4)
        
        self.conv2_bn = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 25, 3)
        self.pool3 = nn.AvgPool2d(4, 4)
        
        self.conv3_bn = nn.BatchNorm2d(25)
        self.conv4 = nn.Conv2d(25, 36, 3)
        self.pool4 = nn.AvgPool2d(2 , 2)
        
        self.fc = nn.Linear(324, 4)

    def forward(self, x):
        x = self.bn0(x)
        x = self.conv1_bn(self.pool1(F.relu(self.conv1(x))))
        x = self.conv2_bn(self.pool2(F.relu(self.conv2(x))))
        x = self.conv3_bn(self.pool3(F.relu(self.conv3(x))))
        x = self.pool4(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc(x)
        return x


model = Net()
model = model.to(device)

In [ ]:
model.eval()

## Sloj po sloj!

#### Prvi (1.) (konvolucijski)

In [ ]:
def lucent_show_layer(layer, n_channels,
                      param_f=None, transforms=None,
                      optimizer=None, preprocess=True, image_size=128):
    n_row = floor( sqrt( n_channels ) )
    n_col = floor( sqrt( n_channels ) )
    _, axs = plt.subplots(n_row, n_col, figsize=(17.55, 18))
    axs = axs.flatten()
    for ix, ax in zip(range(n_row*n_col), axs):
        img = render.render_vis(model, f"{layer}:{ix}", param_f=param_f,
                                transforms=transforms, preprocess=preprocess, progress=False, show_image=False)[0]
        img = np.reshape(img, (image_size, image_size, 3))
        ax.imshow(img)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.margins(x=0, y=0, tight=True)

    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()

In [ ]:
lucent_show_layer('conv1', 9)

#### Drugi (2.)

In [ ]:
lucent_show_layer('conv2', 16)

#### Treći (3.)

In [ ]:
lucent_show_layer('conv3', 25)

# Captum

In [ ]:
!git clone https://github.com/pytorch/captum
%cd captum
!git checkout "optim-wip"
!pip3 --quiet install -e .
import sys
sys.path.append('/content/captum')
%cd ..


In [ ]:
import captum.optim as optimviz
import torchvision

In [ ]:
from typing import Callable, Iterable, Optional

In [ ]:
def vis_neuron_large(
    # funkcija adaptirana iz:
    # https://colab.research.google.com/drive/1Zv7w03hOHfBWaEDMpSR1MA4D6IpAwZln
    model: torch.nn.Module, target: torch.nn.Module, channel: int
) -> None:
    image = optimviz.images.NaturalImage((640, 640)).to(device)
    transforms = torch.nn.Sequential(
        torch.nn.ReflectionPad2d(2),
        optimviz.transforms.RandomSpatialJitter(8),
        optimviz.transforms.RandomScale(scale=(2.15, 1.85, 2, 1.95, 2.05)),
        torchvision.transforms.RandomRotation(degrees=(-15, 15)),
        optimviz.transforms.RandomSpatialJitter(64),
        optimviz.transforms.CenterCrop((640, 640)),
    )
    loss_fn = optimviz.loss.NeuronActivation(target, channel)
    obj = optimviz.InputOptimization(model, loss_fn, image, transforms)
    history = obj.optimize(optimviz.optimization.n_steps(512, False))
    return image()

In [ ]:
def visualize_layer_captum(layer, grid_dim):
    n_row = grid_dim
    n_col = grid_dim
    _, axs = plt.subplots(n_row, n_col, figsize=(19.55, 20))
    axs = axs.flatten()
    for ix, ax in zip(range(n_row*n_col), axs):
        img = vis_neuron_large(model, layer, ix)
        img = img.permute(0, 2, 3, 1)
        with torch.no_grad():
            img = img.cpu().numpy()
        img = img.reshape((640,640,3))
        ax.imshow(img)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.margins(x=0, y=0, tight=True)


    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()

In [ ]:
visualize_layer_captum(model.conv1, 3)

In [ ]:
visualize_layer_captum(model.conv2, 4)

In [ ]:
visualize_layer_captum(model.conv3, 5)

In [ ]:
visualize_layer_captum(model.conv4, 6)